In [1]:
%matplotlib inline
import matplotlib
import matplotlib.patches as mpatches
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("dark")
plt.rcParams['figure.figsize'] = 16, 12
import pandas as pd
from tqdm import tqdm_notebook
import io
from PIL import Image
from glob import glob
from collections import defaultdict
import os
import pickle
from io import BytesIO
import random
import itertools as it

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.datasets import ImageFolder

from kaggle_camera_model_id_lib.utils import PechkaBot, ImageList, NpzFolder, NCrops, TifFolder, TifFolderExFiles
from kaggle_camera_model_id_lib.models import VggHead, StyleVggHead, IEEEfcn, ResNetFC, ResNetX, FatNet1
from kaggle_camera_model_id_lib.models import InceptionResNetV2fc, InceptionResNetV2fcSmall
from kaggle_camera_model_id_lib.utils import jpg_compress, equalize_v_hist, hsv_convert
from kaggle_camera_model_id_lib.utils import scale_crop_pad, gamma_correction
from kaggle_camera_model_id_lib.utils import patch_quality_dich, n_random_crops, n_pseudorandom_crops
from kaggle_camera_model_id_lib.models import DANet, ResNetFeatureExtractor, AvgFcClassifier, FCDiscriminator
from kaggle_camera_model_id_lib.models import AvgClassifier

from sklearn.manifold import TSNE

from scipy.spatial.distance import pdist, squareform

import imgaug as ia
from imgaug import augmenters as iaa

In [2]:
model_factory = {
    'Vgg19Head_E_2b_bn': lambda n_classes: VggHead(num_classes=n_classes, vgg_key='E_2b', load_vgg_bn=True, batch_norm=True),
    'Vgg19Head_E_3b_bn': lambda n_classes: VggHead(num_classes=n_classes, vgg_key='E_3b', load_vgg_bn=True, batch_norm=True),
    'Vgg19Head_E_bn': lambda n_classes: VggHead(num_classes=n_classes, load_vgg_bn=True, vgg_key='E', batch_norm=True),
    'Vgg11Head_A_bn': lambda n_classes: VggHead(num_classes=n_classes, load_vgg_bn=True, vgg_key='A', batch_norm=True),
    'Vgg11Head_A': lambda n_classes: VggHead(num_classes=n_classes, load_vgg_bn=True, vgg_key='A', batch_norm=False),
    'StyleVggHead_bn': lambda n_classes: StyleVggHead(num_classes=n_classes, load_vgg_bn=True),
    'IEEEfcn': lambda n_classes: IEEEfcn(n_classes),
    'resnet18fc_pretrained': lambda n_classes: ResNetFC(
        models.resnet.BasicBlock, [2, 2, 2, 2], num_classes=n_classes, load_resnet='resnet18'),
    'resnet18fc': lambda n_classes: ResNetFC(
        models.resnet.BasicBlock, [2, 2, 2, 2], num_classes=n_classes, load_resnet=None),
    'resnet18X_pretrained': lambda n_classes: ResNetX(
        models.resnet.BasicBlock, [2, 2, 2, 2], num_classes=n_classes, load_resnet='resnet18'),
    'InceptionResNetV2fc_5_10_4': lambda n_classes: InceptionResNetV2fc(
        num_classes=n_classes, nun_block35=5, num_block17=10, num_block8=4),
    'InceptionResNetV2fcSmall_5_10': lambda n_classes: InceptionResNetV2fcSmall(
        num_classes=n_classes, nun_block35=5, num_block17=10),
    'resnet34fc_pretrained': lambda n_classes: ResNetFC(
        models.resnet.BasicBlock, [3, 4, 6, 3], num_classes=n_classes, load_resnet='resnet34'),
    'resnet50fc_pretrained': lambda n_classes: ResNetFC(
        models.resnet.Bottleneck, [3, 4, 6, 3], num_classes=n_classes, load_resnet='resnet50')
}

model_factory_gan = {
    'resnet34_fe': lambda: ResNetFeatureExtractor(models.resnet.BasicBlock, [3, 4, 6, 3], load_resnet='resnet34'),
    'AvgFcClassifier': lambda n_classes: AvgFcClassifier(n_classes),
    'FCDiscriminator': lambda: FCDiscriminator(),
    'AvgClassifier512': lambda n_classes: AvgClassifier(n_classes, 512)
}

In [3]:
model_path = '/home/mephistopheies/storage2/data/camera-model-id/models/resnet34fc/gan/FCDiscriminator_AvgClassifier512/var2/checkpoint.tar'

model_type_fe = 'resnet34_fe'
model_type_d = 'FCDiscriminator'
model_type_c = 'AvgClassifier512'

batch_size = 15

n_classes = 10
crop_size = 256
step = 128
num_workers = 1

to_tensor = transforms.ToTensor()
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

model = DANet(
        model_factory_gan[model_type_fe](),
        model_factory_gan[model_type_d](),
        model_factory_gan[model_type_c](n_classes))


checkpoint = torch.load(model_path)
state = checkpoint['model']
model.load_state_dict(state)
class_to_idx = checkpoint['class_to_idx']
idx2class = dict([(v, k) for (k, v) in class_to_idx.items()])
epoch_log = checkpoint['trainin_log']
for k, v in sorted(epoch_log[-1].items(), key=lambda t: t[0]):
    print('  %s: %0.6f\n' % (k, v))
del(checkpoint)
model = model.cuda()
model = model.eval()

  acc_train_c: 0.995298

  acc_train_d: 0.995153

  acc_train_g: 0.998333

  acc_val: 0.974519

  loss_train_c: 0.014187

  loss_train_d: 0.014571

  loss_train_g: 0.007592

  loss_val: 0.086873

  time: 536.591299



In [4]:
transform=transforms.Compose([
    transforms.Lambda(lambda img: NCrops(np.array(img), crop_size=crop_size, step=step)),
    transforms.Lambda(lambda crops: torch.stack([normalize(to_tensor(crop)) for crop in crops]))
])

In [5]:
train_files = glob('/home/mephistopheies/storage2/data/camera-model-id/svm_data/train_img/*/*.npz')
print(len(train_files))

test_files = glob('/home/mephistopheies/storage2/data/camera-model-id/pseudo_labels/resnet34fc__256_pretrained_random_aug_kaggle_10__phase1/*/*.npz')
print(len(test_files))

49500
1602


In [6]:
out_dir = '/home/mephistopheies/storage2/data/camera-model-id/svm_data/features/train_original/'

for fname in tqdm_notebook(train_files):
    c = fname.split('/')[-2]
    img = np.load(fname)['data']
    X_var = Variable(transform(img).cuda(), volatile=True)
    f = model.feature_exctractor(X_var)
    
    f_mean = f.view(f.shape[0], f.shape[1], -1).mean(dim=2).mean(dim=0).cpu().data.numpy()
    f_max = f.view(f.shape[0], f.shape[1], -1).mean(dim=2).max(dim=0)[0].cpu().data.numpy()
    
    cdir = os.path.join(out_dir, c)
    if not os.path.isdir(cdir):
        os.makedirs(cdir)
    
    np.savez(
        os.path.join(cdir, os.path.basename(fname)), 
        f_mean=f_mean,
        f_max=f_max
    )

In [7]:
out_dir = '/home/mephistopheies/storage2/data/camera-model-id/svm_data/features/train_pseudo/'

for fname in tqdm_notebook(test_files):
    c = fname.split('/')[-2]
    img = np.load(fname)['data']
    X_var = Variable(transform(img).cuda(), volatile=True)
    f = model.feature_exctractor(X_var)
    
    f_mean = f.view(f.shape[0], f.shape[1], -1).mean(dim=2).mean(dim=0).cpu().data.numpy()
    f_max = f.view(f.shape[0], f.shape[1], -1).mean(dim=2).max(dim=0)[0].cpu().data.numpy()
    
    cdir = os.path.join(out_dir, c)
    if not os.path.isdir(cdir):
        os.makedirs(cdir)
    
    np.savez(
        os.path.join(cdir, os.path.basename(fname)), 
        f_mean=f_mean,
        f_max=f_max
    )

In [8]:
real_test_files = glob('/home/mephistopheies/storage2/data/camera-model-id/raw/test/no_class/*.*')

out_dir = '/home/mephistopheies/storage2/data/camera-model-id/svm_data/features/test/'

def loader(path):
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')
    
    
for fname in tqdm_notebook(real_test_files):
    img = np.array(loader(fname))
    X_var = Variable(transform(img).cuda(), volatile=True)
    f = model.feature_exctractor(X_var)
    
    f_mean = f.view(f.shape[0], f.shape[1], -1).mean(dim=2).mean(dim=0).cpu().data.numpy()
    f_max = f.view(f.shape[0], f.shape[1], -1).mean(dim=2).max(dim=0)[0].cpu().data.numpy()
    
    np.savez(
        os.path.join(out_dir, os.path.basename(fname)), 
        f_mean=f_mean,
        f_max=f_max
    )